In [ ]:
import shutil
import time

import matplotlib.pyplot as plt
from fsspec.implementations.local import LocalFileSystem
from dask import delayed
from dask.distributed import Client, LocalCluster

from dummy_baskets import generate_dummy_baskets, run_index_basket_upload_test
from weave.index.index_pandas import IndexPandas
from weave.index.index_sqlite import IndexSQLite
from weave import Pantry

In this notebok we will run 5 upload tests on each index backend (IndexPandas, IndexSQLite, IndexSQL) and compare the average upload time. We will be using 1000 baskets, 5 upload files, 1 MB file size, and 5 basket types as the parameters in each test. Statistical tests will be performed across the three different index backends to determine if there is a statistical significant difference in upload time between all the index backends.